<a href="https://colab.research.google.com/github/srikarraju/eGrocery_Demand_Prediction/blob/main/Abuse_Detection_Indic_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers

In [ ]:
pip install sentencepiece

In [ ]:
#READING TRAININ DATA FILE
import pandas as pd

train_df = pd.read_csv('/content/drive/MyDrive/MultiLingual_Abuse_Detection/Data/clean_englis_train_data.csv')
val_df = pd.read_csv('/content/drive/MyDrive/MultiLingual_Abuse_Detection/Data/clean_englis_val_data.csv')

In [ ]:
train_texts,val_texts = [],[]
max_length = 128

train_labels, val_labels = [], []

for i in range(len(train_df)):
  train_texts.append(train_df['commentText'].iloc[i])
  train_labels.append(int(train_df['label'].iloc[i]))

for i in range(len(val_df)):
  val_texts.append(val_df['commentText'].iloc[i])
  val_labels.append(int(val_df['label'].iloc[i]))

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('ai4bharat/indic-bert')
model = AutoModelForSequenceClassification.from_pretrained('ai4bharat/indic-bert',num_labels=2).to('cuda')

Downloading:   0%|          | 0.00/507 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.38M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/129M [00:00<?, ?B/s]

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertForSequenceClassification: ['predictions.LayerNorm.weight', 'predictions.decoder.bias', 'predictions.dense.weight', 'sop_classifier.classifier.weight', 'predictions.decoder.weight', 'predictions.dense.bias', 'predictions.LayerNorm.bias', 'predictions.bias', 'sop_classifier.classifier.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indi

In [ ]:
max_length = 128
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)
valid_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=max_length)

In [ ]:
import torch
class AbuseDetetctionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

# convert our tokenized data into a torch Dataset
train_dataset = AbuseDetetctionDataset(train_encodings, train_labels)
valid_dataset = AbuseDetetctionDataset(valid_encodings, val_labels)

In [ ]:
from sklearn.metrics import f1_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  acc = f1_score(labels, preds)
  return {
      'accuracy': acc,
  }

In [ ]:
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir='./',          # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=3000,               # log & save weights each logging_steps
    evaluation_strategy="steps",     # evaluate each `logging_steps`
    save_steps = 3000
)

In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,          # evaluation dataset
    compute_metrics=compute_metrics    # the callback that computes metrics of interest
)

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 549145
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 17161


Step,Training Loss,Validation Loss,Accuracy
3000,0.503100,0.406060,0.807077
6000,0.393500,0.381775,0.823996
9000,0.367000,0.361586,0.832990
12000,0.355900,0.348217,0.844121
15000,0.344400,0.340416,0.847556


***** Running Evaluation *****
  Num examples = 96913
  Batch size = 32
Saving model checkpoint to ./checkpoint-3000
Configuration saved in ./checkpoint-3000/config.json
Model weights saved in ./checkpoint-3000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 96913
  Batch size = 32
Saving model checkpoint to ./checkpoint-6000
Configuration saved in ./checkpoint-6000/config.json
Model weights saved in ./checkpoint-6000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 96913
  Batch size = 32
Saving model checkpoint to ./checkpoint-9000
Configuration saved in ./checkpoint-9000/config.json
Model weights saved in ./checkpoint-9000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 96913
  Batch size = 32
Saving model checkpoint to ./checkpoint-12000
Configuration saved in ./checkpoint-12000/config.json
Model weights saved in ./checkpoint-12000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 96913
  Batch size = 32
Saving mo

TrainOutput(global_step=17161, training_loss=0.3858398202763078, metrics={'train_runtime': 8221.7198, 'train_samples_per_second': 66.792, 'train_steps_per_second': 2.087, 'total_flos': 3280880772748800.0, 'train_loss': 0.3858398202763078, 'epoch': 1.0})

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 46077
  Batch size = 32


{'epoch': 5.0,
 'eval_accuracy': 0.6893962198387175,
 'eval_loss': 0.6583058834075928,
 'eval_runtime': 122.8106,
 'eval_samples_per_second': 375.187,
 'eval_steps_per_second': 11.725}

In [ ]:
model_path = "/content/drive/MyDrive/MultiLingual_Abuse_Detection/Output/indic-transformers_clean_english_epoch_1.csv"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

Configuration saved in /content/drive/MyDrive/MultiLingual_Abuse_Detection/Output/indic-transformers_clean_english_epoch_1.csv/config.json
Model weights saved in /content/drive/MyDrive/MultiLingual_Abuse_Detection/Output/indic-transformers_clean_english_epoch_1.csv/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/MultiLingual_Abuse_Detection/Output/indic-transformers_clean_english_epoch_1.csv/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/MultiLingual_Abuse_Detection/Output/indic-transformers_clean_english_epoch_1.csv/special_tokens_map.json


('/content/drive/MyDrive/MultiLingual_Abuse_Detection/Output/indic-transformers_clean_english_epoch_1.csv/tokenizer_config.json',
 '/content/drive/MyDrive/MultiLingual_Abuse_Detection/Output/indic-transformers_clean_english_epoch_1.csv/special_tokens_map.json',
 '/content/drive/MyDrive/MultiLingual_Abuse_Detection/Output/indic-transformers_clean_english_epoch_1.csv/spiece.model',
 '/content/drive/MyDrive/MultiLingual_Abuse_Detection/Output/indic-transformers_clean_english_epoch_1.csv/added_tokens.json',
 '/content/drive/MyDrive/MultiLingual_Abuse_Detection/Output/indic-transformers_clean_english_epoch_1.csv/tokenizer.json')

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/MultiLingual_Abuse_Detection/Output/indic_bert_1_epoch')
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/MultiLingual_Abuse_Detection/Output/indic_bert_1_epoch',num_labels=2)

In [ ]:
import pandas as pd
test_df = pd.read_csv('/content/drive/MyDrive/MultiLingual_Abuse_Detection/Data/test_data.csv')
test_sentences = []
for i in range(len(test_df)):
  test_sentences.append(test_df['commentText'].iloc[i])
print(len(test_sentences))

74253


In [ ]:
outputs = []
for i in range(100):
  print(i)
  test_encodings = tokenizer(val_texts[i], truncation=True, padding=True, max_length=128,return_tensors='pt')
  tesnor_op = model(**test_encodings)
  probs = tesnor_op[0].softmax(1)
  outputs.append(probs.argmax())

In [ ]:
for i in range(100):
  if outputs[i].item()!= val_labels[i]:
    print(val_texts[i], val_labels[i],outputs[i].item())

हराम के पंद्रह लाख पाने का इंतजार करने वालों को कोई खुश नहीं कर सकता जिस तरह पच्चीस तीस हजार रुपए  प्राइवेट काम करके कमाने वाले की कोई बेरोजगारी नहीं दूर कर सकता 0 1
Waste Gadu Vadi mede poste telustundi Vedini na road meda kalachali 0 1
പ്രണയിച്ചവൻ നീ പോടാ കള്ള താടി 😝 1 0
I mean nuvu koda  kukka 1 0
"Mujhe pgal kutte kaat liya isiliye likhwa rha hun"😂 0 1
choojaa bho bho bho bho 🤣 0 1
- 🚩🚩🚩🚩 Acha sale jutlye is tere baap ki bhi lash thi kya jo itna jal rahi hai teri 0 1
Bina madar  la 0 1
right aapne Jo kha sahi kha log Haram zade h kuutte h nhi dekhni to na dekhe 0 1
𝘽𝙝𝙖𝙪𝙟𝙖.  𝙋𝙪𝙧𝙖 . 𝙎𝙚𝙭𝙮 𝘼𝙘𝙝𝙞. 𝙋𝙪𝙧𝙖.𝙃𝙖𝙧𝙙 𝙘𝙤𝙧𝙚 . 𝘽𝙥 𝙃𝙖𝙗𝙖 . 𝘼𝙪.  𝙆𝙚𝙩𝙚 𝙣𝙖𝙗𝙖 𝘽𝙝𝙖𝙪𝙟𝙖 . 2𝙂𝙝𝙖𝙣𝙩𝙖 .𝙠𝙪 𝙠𝙖𝙝𝙪𝙣𝙖. 1 0
creation niyamma antaventi 😠😠😠 1 0
Top fullgaa vundigaa enduku malli etti choopistnaav meeda chunni vesta manchi comments vastaay choopinchaavanukoo vastaavaa ani adugutaaru avasaramaa neeku daridrapu mohan daanaaa ikanayinaa kappukoo avi vunnay ani cgoopinchaku chastaav 1 0
Chor ahet he sagle 1 0
ജ്ജ് തനി കച്ചറ കൂതറ മാന

In [ ]:
result_Indic_bert_1_epoch_val = pd.DataFrame()
result_Indic_bert_1_epoch_val['commentText'] = val_texts[0:40820]
result_Indic_bert_1_epoch_val['Expected'] = val_labels[0:40820]
result_Indic_bert_1_epoch_val['Predicted'] = outputs
result_Indic_bert_1_epoch_val.to_csv('/content/drive/MyDrive/MultiLingual_Abuse_Detection/Output/result_Indic_bert_1_epoch_val',index=False)

In [ ]:
formatted_outputs = []
for i in range(len(result_Indic_bert_1_epoch)):
  formatted_outputs.append(result_Indic_bert_1_epoch['Expected'].iloc[i].item())
print(formatted_outputs)

[0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 

In [ ]:
result_Indic_bert_1_epoch['Expected'] = formatted_outputs
result_Indic_bert_1_epoch.to_csv('/content/drive/MyDrive/MultiLingual_Abuse_Detection/Output/result_Indic_bert_1_epoch',index=False)

In [ ]:
model_path = "/content/drive/MyDrive/MultiLingual_Abuse_Detection/Output/indic_bert_1_epoch"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

Configuration saved in /content/drive/MyDrive/MultiLingual_Abuse_Detection/Output/indic_bert_1_epoch/config.json
Model weights saved in /content/drive/MyDrive/MultiLingual_Abuse_Detection/Output/indic_bert_1_epoch/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/MultiLingual_Abuse_Detection/Output/indic_bert_1_epoch/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/MultiLingual_Abuse_Detection/Output/indic_bert_1_epoch/special_tokens_map.json


('/content/drive/MyDrive/MultiLingual_Abuse_Detection/Output/indic_bert_1_epoch/tokenizer_config.json',
 '/content/drive/MyDrive/MultiLingual_Abuse_Detection/Output/indic_bert_1_epoch/special_tokens_map.json',
 '/content/drive/MyDrive/MultiLingual_Abuse_Detection/Output/indic_bert_1_epoch/spiece.model',
 '/content/drive/MyDrive/MultiLingual_Abuse_Detection/Output/indic_bert_1_epoch/added_tokens.json',
 '/content/drive/MyDrive/MultiLingual_Abuse_Detection/Output/indic_bert_1_epoch/tokenizer.json')